# TensorFlow Category 5
> Sequence (시계열) 데이터 다루기

- toc:true
- branch: master
- badges: true
- comments: false
- author: 최서연
- categories: [TensorFlow]

문제 5. Sequences(sunspots)

type A
```python
tf.keras.layers.Lambda(lambda x: x + 400)
```
lambda를 사용하여 sequence에 사용하여 문제가 제시됨(건들지 말라고 문제에서 언급될 것)

type B
```python
min = np.min(series)
max = np.max(series)
seroes -= min
seroes /= max
time = np.array(time_step)
```

[0,1,2,3,4,5,6,7,8,9]

window_size = 3, shift = 1
- 3일의 데이터(0,1,2)를 가지고 타겟(예측값인 3) 단일 예측할거야
- 3일의 데이터(1,2,3)를 가지고 타겟(예측값인 4) 단일 예측할거야
- ...

window_size = 5, shift = 2
- 5일의 데이터(0,1,2,3,4)를 가지고 타겟(예측값인 5) 단일 예측할거야
- 5일의 데이터(2,3,4,5,6)를 가지고 타겟(예측값인 7) 단일 예측할거야
- ...

5번 문제는 many to many문제 
- 학습 데이터(0,1,2,3,4)가지고 타겟(예측값1,2,3,4,5) sequence 예측,
- window_size = 5

5번 문제는 many to many문제 가능_하지만 시험에서는 위에 문제 나옴
- 학습 데이터(0,1,2,3,4)가지고 타겟(예측값5,6,7,8,9) sequence 예측,
- window_size = 5

$\star$ many to many 기법$\star$ 

drop_remainder = False
- False로 줘야 window size에 안맞는 예들이 사라짐
- [0,1,2,3,4,5]있을때 0,1,2,가지고 shift 1일때 3,4,5 예측 후 (4,5) 혹은 (5) 이렇게 입력값과 출력값이 many to many의 수가 같지 않은 것 사라짐

buffer_size = 30,000(full_shuffle)
- 0~29,999 셔플

buffer_size = 1,000
- 0~999 셔플
-999~1,999 셔플
- 1,999~2,999 셔플
- ...
- 28,999~29,999 셔플

```python
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
	series = tf.expand_dims(series, axis=-1) # 차원을 1차원에서 2차원으로 늘려줌
	ds = tf.data.Dataset.from_tensor_slices(series) # dataset으로 변환 아래 옵션 사용위해 데이터셋으로서의 변환이 필요
	ds = ds.window(window_size + 1, shift = 1, drop_remainder=True) # 1을 더해주는 이유는 window 크기가 y의 끝까지 다 포함되도록 하기 위해서, 만약 (1,2,3)으로 (4,5,6) 예측하면 window_size + 3이 되겠지. 지금은 (1,2,3)으로 (2,3,4)예측하니까 window_size+1임
	ds = flat_map(lambda w: w.batch(sindow_size + 1)) # 데이터가 [[1],[2],[3],[4],[5]]이렇게 묶이는데 flat_map쓰면 [1,2,3,4,5] 펼쳐준다. window_size는 위와 같게함 여기서는 +1으로서 같게 함
	ds = ds.shuffle(shuffle_buffer)
	ds = ds.map(lambda w : (w[:-1],w[1:])) # x,y데이터 분할 지금 window_size+1놓았을때 window_size가 5여서 총 6이 입력된다면, 예를들어 [0,1,2,3,4,5]이렇게 6개 있다치면 x는 -1번째까지, [0,1,2,3,4]가 되고, y는 [1,2,3,4,5]가 된다. 
	# 만약 [0,1,2,3,4,5]에서 x가[0,1,2,3,4]이고 y로서 5만 예측하고 싶다면 (2[:-1].w[-1])해주면 된다.(many to one) 보토 주가예측 모델 만들때
	return ds.batch(batch_size).prefetch(1) # 하나의 배치를 미리 더 만들어라, 미리 만들어 놓기 때문에 배치가 실행되는 속도가 빨라진다.
```

Conc1D
- 1차원 합성곱
- kernal_size는 단일 데이터
- strides는 몇 칸씩 이동하며 학습할 건지 정하는 역할
- padding = 'causal' 원인을 남겨둔다 이해하면 될 듯 앞의 결과를 남겨두기 위한 causal옵션

Stochastic Gradient Descent(SGD)
- 경사하강법 : 경사를 타고 내려와 최저점(loss가 가장 낮은 지점)에 도달하는 최적화 알고리즘
- 학습율이 너무 작은 경우 학습이 진행되지 않는다.
- 학습율이 너무 큰 경우 minimum에 도달하지 않는다.

Momentum(관성)
- momentum = 0.0 마찰력이 높다는 뜻. local minimum value에 수렴하는 오류가 생긴다. global minimun value에 수렴해야 하는데!
- momentum = 0.9로 보통 놓음 마찰력을 줄이는 역할을 한다.
	- adam은 기본적으로 마찰력 낮춰져 있음

Optimizer
```python
optimizer = tf.keras.optimizers.SGD(lr = 0.0001,momentum=0.9)
model.compile(loss = loss, optimizer, metrics["mae"]
```

Huber Loss
- 어떤 특정 임계값 기준으로
- 오차가 0에 가까울수록 MSE(제곱평균오차)
- 오차가 0에서 멀어질수록 MAE(절대값평균오차)
- 두 loss가 섞여 있음!!!

## 확인

1. GPU 옵션 켜져 있는지 확인할 것!!! (수정 - 노트설정 - 하드웨어설정 (GPU))

## 순서

1. **import**: 필요한 모듈 import
2. **전처리**: 학습에 필요한 데이터 전처리를 수행합니다.
3. **모델링(model)**: 모델을 정의합니다.
4. **컴파일(compile)**: 모델을 생성합니다.
5. **학습 (fit)**: 모델을 학습시킵니다.

## 문제

For this task you will need to train a neural network
to predict sunspot activity using the Sunspots.csv
provided. 

Your neural network is expected to have an MAE
of at least 20, with top marks going to one with an MAE
of around 15. 

At the bottom is provided some testing
code should you want to check before uploading which measures
the MAE for you. 

Strongly recommend you test your model with
this to be able to see how it performs.



-------------------------------
**Sequence(시퀀스)**

Sunspots.csv를 사용하여 **태양 흑점 활동(sunspot)**을 예측하는 인공신경망을 만듭니다.

MAE 오차 기준으로 최소 20이하로 예측할 것을 권장하며, 탑 랭킹에 들려면 MAE 15 근처에 도달해야합니다.

아래 주어진 샘플코드는 당신의 모델을 테스트 하는 용도로 활용할 수 있습니다.

-----------------------------------


## 필요한 모듈 import

In [ ]:
import csv
import tensorflow as tf
import numpy as np
import urllib

from tensorflow.keras.layers import Dense, LSTM, Lambda, Conv1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import Huber

In [ ]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/Sunspots.csv'
urllib.request.urlretrieve(url, 'sunspots.csv')

## csv 파일로부터 데이터셋 만들기

csv.reader() 함수를 활용합니다.

* 첫번째 파라미터에는 file을 , delimiter에는 구분자를 넣어 줍니다.

In [ ]:
with open('sunspots.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    i = 0
    for row in reader:
        print(row)
        i+=1
        if i > 10:
            break

빈 list를 만들어 줍니다. (sunspots, time_step)

In [ ]:
sunspots = []
time_step = []

`time_step`에는 **index** 값을, `sunspots`에는 sunspots의 정보를 넣어 줍니다.

In [ ]:
with open('sunspots.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    # 첫 줄은 header이므로 skip 합니다.
    next(reader)
    for row in reader:
        sunspots.append(float(row[2]))
        time_step.append(int(row[0]))

sunspots, time_step 확인

In [ ]:
sunspots[:5]

In [ ]:
time_step[:5]

sunspots와 time_step을 `numpy array`로 변환합니다.

* 참고: 모델은 list 타입을 받아들이지 못합니다. 따라서, numpy array 로 변환해 줍니다.

In [ ]:
series = np.array(sunspots)
time = np.array(time_step)

In [ ]:
series.shape, time.shape

## 태양의 흑점 활동 (sunspots) 시각화

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(16, 8))
plt.plot(time, series)
plt.xlabel("Time")
plt.ylabel("Value")
plt.grid(True)

## Train Set, Validation Set 생성

3000 인덱스를 기준으로 Train / Validation Set를 분할 합니다.

In [ ]:
split_time = 3000

In [ ]:
time_train = time[:split_time]
time_valid = time[split_time:]

x_train = series[:split_time]
x_valid = series[split_time:]

## Window Dataset Loader

자세한 Dataset 활용법은 [블로그 링크](https://teddylee777.github.io/tensorflow/dataset-batch-window)를 참고해 보시고, 연습해보세요!

In [ ]:
# 윈도우 사이즈
window_size=30
# 배치 사이즈
batch_size = 32
# 셔플 사이즈
shuffle_size = 1000

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

`train_set`와 `validation_set`를 만듭니다.

In [ ]:
train_set = windowed_dataset(x_train, 
                             window_size=window_size, 
                             batch_size=batch_size,
                             shuffle_buffer=shuffle_size)

validation_set = windowed_dataset(x_valid, 
                                  window_size=window_size,
                                  batch_size=batch_size,
                                  shuffle_buffer=shuffle_size)

## 모델 정의 (Sequential)

In [ ]:
from IPython.display import Image

Image('https://i.stack.imgur.com/NmYZJ.png')

In [ ]:
model = Sequential([
    tf.keras.layers.Conv1D(60, kernel_size=5,
                         padding="causal",
                         activation="relu",
                         input_shape=[None, 1]),
    tf.keras.layers.LSTM(60, return_sequences=True),
    tf.keras.layers.LSTM(60, return_sequences=True),
    tf.keras.layers.Dense(30, activation="relu"),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x: x * 400)
])

모델의 구조 요약을 확인합니다.

In [ ]:
model.summary()

## 컴파일 (compile)

**Optimizer**는 SGD(Stochastic Gradient Descent) 를 사용합니다.

* lr(learning_rate): 학습률입니다.
* momentum: 모멘텀 (가중치) 입니다.

In [ ]:
optimizer = SGD(lr=1e-5, momentum=0.9)

**Huber Loss**: MSE와 MAE를 절충한 후버 손실(Huber loss)

\begin{split}L_{\delta}=\left\{\begin{matrix}
\frac{1}{2}(y - \hat{y})^{2} & if \left | (y - \hat{y})  \right | < \delta\\
\delta ((y - \hat{y}) - \frac1 2 \delta) & otherwise
\end{matrix}\right.\end{split}

In [ ]:
loss= Huber()

model.compile()시 우리가 튜닝한 **optimizer**와 **loss**를 활용합니다.

In [ ]:
model.compile(loss=loss,
              optimizer=optimizer,
              metrics=["mae"])

## ModelCheckpoint: 체크포인트 생성

`val_loss` 기준으로 epoch 마다 최적의 모델을 저장하기 위하여, ModelCheckpoint를 만듭니다.
* `checkpoint_path`는 모델이 저장될 파일 명을 설정합니다.
* `ModelCheckpoint`을 선언하고, 적절한 옵션 값을 지정합니다.

In [ ]:
checkpoint_path = 'tmp_checkpoint.ckpt'
checkpoint = ModelCheckpoint(checkpoint_path, 
                             save_weights_only=True, 
                             save_best_only=True, 
                             monitor='val_mae',
                             verbose=1)

## 학습 (fit)

In [ ]:
epochs=100

In [ ]:
history = model.fit(train_set, 
                    validation_data=(validation_set), 
                    epochs=epochs, 
                    callbacks=[checkpoint],
                   )

## 학습 완료 후 Load Weights (ModelCheckpoint)

학습이 완료된 후에는 반드시 `load_weights`를 해주어야 합니다.

그렇지 않으면, 열심히 ModelCheckpoint를 만든 의미가 없습니다.

In [ ]:
model.load_weights(checkpoint_path)

## 학습 오차에 대한 시각화

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(12, 9))
plt.plot(np.arange(1, epochs+1), history.history['loss'])
plt.plot(np.arange(1, epochs+1), history.history['val_loss'])
plt.title('Loss / Val Loss', fontsize=20)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['loss', 'val_loss'], fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(12, 9))
plt.plot(np.arange(1, epochs+1), history.history['mae'])
plt.plot(np.arange(1, epochs+1), history.history['val_mae'])
plt.title('MAE / Val MAE', fontsize=20)
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend(['mae', 'val_mae'], fontsize=15)
plt.show()